In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from itertools import combinations
from sklearn.decomposition import PCA
import scipy

In [2]:
ph=1.5574E-4
times=[1,2,3,6,7,9,13,16,21,24,31] 
pxt=[(x) for x in list(range(3,50,1))]
neh=range(1,120)
path="C:/Workplace/C++/d2ome_v2/v2/v2/bin/Debug/liverpool_liver_31_test/"

In [17]:
all_AAs=['a','w',  'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'p','q', 'r', 's', 't', 'v', 'y']
def charcount(pepSeq):
    res={key: 0 for key in all_AAs}     
    for c in pepSeq.lower():
        if c.isalpha():
            res[c]+=1
    return res

In [18]:
def getPeptiedsInfo(protien,time):
    
    peptides=[x.split('_1_')[0] for x in os.listdir(path) if ((protien+".csv" in x) and ("_1_" in x)) ]
    peptides=[x  for x in peptides if 'A' in x]
#     all_AAs=np.unique([c for c in ("".join(sorted([x.replace('_','').replace('2','').replace('3','') for x in peptides])).lower())])
    peptidesinfo=dict()
    len(peptides)
    

    for k in range(len(peptides)):
        data=pd.read_csv(f"{path}{peptides[k]}_{time}_{protien}.csv")
        data["pxt_neh"]=data.NEH*data.pxt
        data=data[["NEH","pxt","RMSE","pxt_neh","T_NEH","NH"]] 
        minval=min(data.RMSE)

        data=data[data.RMSE==minval]

        peptidesinfo[peptides[k]]={"pxt_neh":float(data.pxt_neh),"T_NEH":float(data.T_NEH),"Seq":peptides[k],
                               "charcount":charcount(peptides[k])}
        
    print(len(peptides))    
    visited=set()
    visited.add(peptides[0].split('_')[1])
    coefficients=[]
    for k in range(0,len(peptides)-1):
        if(peptides[k+1].split('_')[1] in visited):
            continue

        pep1=peptidesinfo[peptides[k]]
        pep2=peptidesinfo[peptides[k+1]]    
        visited.add(peptides[k].split('_')[1])

        r=pep1["pxt_neh"]/pep2["pxt_neh"]
        coefficients.append([pep1["charcount"][c]-r*pep2["charcount"][c] for c in all_AAs])
        
    return peptides,peptidesinfo,all_AAs,np.array(coefficients)


In [5]:
def compute_result(peptides,peptidesinfo,all_AAs,sol):
    t_neh,e_neh=[],[]
    for i in range (len(peptides)):
        pep1=peptidesinfo[peptides[i]]
        p_neh=0
        for index,c in enumerate(all_AAs): 
            p_neh+=(pep1["charcount"][c]*sol[index]) 
        t_neh.append(pep1["T_NEH"])
        e_neh.append(p_neh)

    res=pd.DataFrame([peptides,t_neh,e_neh])
    res=res.transpose()
    res.columns=["peptides","True_NEH","e_NEH"]
    res["dif"]=abs(res.True_NEH-res.e_NEH)
#     print(np.mean(res.dif),np.median(res.dif),np.std(res.dif))
    return res.sort_values(by='dif')

In [19]:
time=31
a_val=4
w_val=0
# proteins=["CPSM_MOUSE","FAS_MOUSE","DHB4_MOUSE","ASSY_MOUSE","ATPA_MOUSE","ATPB_MOUSE","BHMT1_MOUSE",
#          "CLH1_MOUSE","DHE3_MOUSE","DHSO_MOUSE","ECHA_MOUSE",
#          "G3P_MOUSE","HMCS2_MOUSE","HYES_MOUSE","MYH9_MOUSE","PYC_MOUSE","PYGL_MOUSE","SAHH_MOUSE",
#          "SARDH_MOUSE","SBP1_MOUSE","THIM_MOUSE","TKT_MOUSE"]
proteins=pd.read_csv("H:/Warehouse/Data/liverpool/liver/6_23_2022_c/"+"Analyzed_Proteins_and_Their_Rates.csv").Proteins.unique()
coefficients=np.array([])
_peptides,_peptidesinfo=[],[]
for protine in proteins:
    try:
        peptides1,peptidesinfo1,all_AAs1,coefficients1=getPeptiedsInfo(protine,time) 
        print(protine,len(all_AAs1)==20)
        if(coefficients.shape[0]==0):
            coefficients=coefficients1
        else:
            coefficients=np.concatenate((coefficients1,coefficients), axis=0)
        _peptides.append(peptides1)
        _peptidesinfo.append(peptidesinfo1)
    except:
        continue
    
sol=list(scipy.optimize.nnls(coefficients[:,2 :],-a_val*coefficients[:,0]-w_val*coefficients[:,1])[0])
# sol=list(scipy.optimize.lsq_linear(coefficients[:,1:],-a_val*coefficients[:,0])[0])
print("====>",len(sol))
sol.insert(0,a_val) 
sol.insert(1,w_val) 
print("\nValues\n","\n".join([all_AAs1[index]+"= "+str(x) for index,x in enumerate(sol)]))
res=pd.DataFrame()
for i in range(len(proteins)):
    res=pd.concat([res,compute_result(_peptides[i],_peptidesinfo[i],all_AAs1,sol)])

print("\nstats",np.mean(res.dif),np.median(res.dif),np.std(res.dif))    
 

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
10
ASSY_MOUSE True
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15
ATPA_MOUSE True
15
ATPB_MOUSE True
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
BHMT1_MOUSE True
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
CLH1_MOUSE True
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
24
CPSM_MOUSE True
0
3
CPT2_MOUSE True
0
0
0
0
0

IndexError: list index out of range

In [7]:
print("proteins",len(proteins),"peptides",res.shape[0])

proteins 22 peptides 176


In [8]:
res=res.sort_values(by='dif')
res.head(20)

,peptides,True_NEH,e_NEH,dif
8,_HALIIYDDLSK_2,17.0,17.007995,0.007995
1,_DATSLNQAALYR_2,27.0,27.009105,0.009105
0,_LEDTLWAGLTDQHVK_3,24.0,24.015663,0.015663
1,_DLEAHIDTANK_2,22.0,21.979315,0.020685
2,_DFTATFGPLDSLNTR_2,22.0,21.974485,0.025515
2,_ALMGLYNGQVLcK_2,20.0,19.933897,0.066103
11,_SLQDIIAILGMDELSEEDK_2,39.0,38.928325,0.071675
3,_AQFGSDLDAATQQLLSR_3,40.0,39.921229,0.078771
2,_AQFGSDLDAATQQLLSR_2,40.0,39.921229,0.078771
21,_TSAcFEPSLDYMVTK_2,23.0,22.914101,0.085899


In [9]:
res.tail(20)

,peptides,True_NEH,e_NEH,dif
5,_HGGTIPVVPTAEFQDR_3,32.0,28.310639,3.689361
2,_DMGMVTILVHNTASALR_3,28.0,24.225979,3.774021
4,_VSQPIEGHAASFAQFK_3,39.0,35.187314,3.812686
2,_PTGGAGAVAMLIGPK_2,29.0,25.068976,3.931024
0,_ARPEFMLPVHFYGR_4,28.0,24.049428,3.950572
14,_VLDSGAPIKIPVGPETLGR_3,34.0,29.999613,4.000387
2,_GFIGPGIDVPAPDMSTGER_2,37.0,32.911594,4.088406
8,_HLGIVGEcNIQFALHPT_3,32.0,27.89675,4.10325
6,_WSScNIFSTQDHAAAAIAK_3,43.0,38.819395,4.180605
1,_DILQDVLDADLSNEAFPF_2,34.0,38.239364,4.239364


In [10]:
temp=pd.DataFrame(coefficients)
temp.columns=all_AAs1
temp 

,a,w,c,d,e,f,g,h,i,k,l,m,n,p,q,r,s,t,v,y
0,1.236842,1.0,0.00000,-1.842105,2.078947,1.000000,1.000000,0.000000,-2.763158,0.000000,3.078947,0.000000,0.078947,-1.763158,-0.921053,0.078947,-0.921053,-0.921053,0.078947,0.000000
1,1.869048,0.0,0.00000,-0.261905,1.000000,0.000000,-1.130952,-1.130952,1.869048,-1.130952,-1.261905,0.000000,1.000000,0.738095,-3.523810,1.000000,-0.130952,1.000000,-0.130952,-1.130952
2,-2.249516,0.0,0.00000,-1.249516,0.000000,-1.624758,-0.624758,1.000000,-2.249516,-2.249516,2.000000,-1.624758,0.000000,2.000000,2.375242,0.000000,1.000000,0.000000,-2.249516,1.000000
3,1.397084,0.0,0.00000,2.000000,-1.205831,1.000000,1.000000,0.000000,0.191253,1.397084,0.000000,0.397084,-0.602916,0.000000,-1.411662,0.000000,-1.205831,0.000000,1.397084,-0.602916
4,-0.429167,0.0,0.00000,-1.429167,0.570833,-1.429167,-1.429167,0.000000,3.000000,-0.429167,0.000000,-0.429167,1.000000,-2.858333,4.000000,0.000000,-2.287500,-2.858333,-3.287500,-0.429167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,-0.400001,0.0,0.00000,0.300000,1.000000,-0.700000,0.300000,1.000000,0.300000,0.300000,-0.400001,0.000000,-0.700000,0.000000,-0.700000,0.000000,-0.700000,-0.100001,-0.400001,0.000000
145,-0.034783,0.0,0.00000,-0.017391,0.000000,-0.017391,-0.017391,0.000000,-0.017391,-0.017391,-0.034783,0.000000,-0.017391,0.000000,-0.017391,0.000000,-0.017391,-0.052174,-0.034783,0.000000
146,0.933210,0.0,-1.06679,-0.066790,-1.066790,-0.066790,1.000000,0.000000,1.000000,-0.066790,0.933210,-1.066790,1.000000,-1.066790,1.000000,0.000000,-1.133581,0.866419,0.933210,-1.066790
147,0.137600,0.0,1.00000,1.000000,0.137600,1.000000,-0.862400,0.000000,-0.862400,0.137600,-0.724800,0.137600,-2.587200,0.137600,-0.862400,0.000000,1.137600,0.275200,-1.587200,1.000000


In [11]:
# for c in range( len(coefficients)):
#     print(np.dot(coefficients[c],sol))